In [28]:
import pandas as pd

# Original dataset
data = {
    'Gene': ['Gene1', 'Gene1', 'Gene1', 'Gene1', 'Gene1', 'Gene1', 'Gene2', 'Gene2', 'Gene2', 'Gene2', 'Gene3', 'Gene3', 'Gene3', 'Gene3'],
    'TF': ['TF1', 'TF1', 'TF1', 'TF2', 'TF3', 'TF3', 'TF2', 'TF2', 'TF3', 'TF3', 'TF1', 'TF3', 'TF3','TF3'],
    'Start_Position': [100, 105, 130, 110, 105, 110, 200, 205, 210, 215, 300, 305, 310, 350],
    'End_Position': [110, 115, 140, 120, 115, 120, 210, 215, 220, 225, 310, 315, 320, 360]
}

df = pd.DataFrame(data)

print(df)

data = {
    'Gene': ['Gene1', 'Gene1', 'Gene1', 'Gene1', 'Gene1', 'Gene1', 'Gene2', 'Gene2', 'Gene2', 'Gene2', 'Gene3', 'Gene3', 'Gene3', 'Gene3','Gene3'],
    'TF': ['TF1', 'TF1', 'TF1', 'TF2', 'TF3', 'TF3', 'TF2', 'TF2', 'TF3', 'TF3', 'TF1', 'TF3', 'TF3', 'TF3','TF3'],
    'Start_Position': [100, 105, 130, 110, 105, 110, 200, 205, 210, 215, 300, 305, 310, 350, 355],
    'End_Position': [110, 115, 140, 120, 115, 120, 210, 215, 220, 225, 310, 315, 320, 360,365],
    'Strand': ['+', '+', '+', '-', '+', '-', '+', '+', '-', '-', '-', '-', '+', '-','+']
}
df = pd.DataFrame(data)

# Save DataFrame as a TSV file
df.to_csv("output_data.tsv", sep='\t', index=False)

# Save DataFrame as an Excel file
df.to_excel("output_data.xlsx", index=False)

In [39]:
# Read the Excel file into a DataFrame
#df_excel = pd.read_excel("fimo.xlsx")
df_excel = pd.read_csv("fimo.tsv",delimiter='\t')
print(df_excel)

       motif_ID motif_ALT_ID             seq_ID  site_Start  site_End  \
0      MA1402.1         BPC6  Glyma.04G061400.1      1936.0    1956.0   
1      MA1402.1         BPC6  Glyma.04G061400.1      1938.0    1958.0   
2      MA1402.1         BPC6  Glyma.04G061400.1      1940.0    1960.0   
3      MA1402.1         BPC6  Glyma.04G061400.1      1942.0    1962.0   
4      MA1402.1         BPC6  Glyma.04G061400.1      1944.0    1964.0   
...         ...          ...                ...         ...       ...   
59173  MA1050.1    OsI_08196  Glyma.08G118200.1      1910.0    1917.0   
59174  MA1050.1    OsI_08196  Glyma.04G238300.1       427.0     434.0   
59175  MA1050.1    OsI_08196  Glyma.16G176700.1       135.0     142.0   
59176  MA1050.1    OsI_08196  Glyma.03G042700.1      1553.0    1560.0   
59177  MA1050.1    OsI_08196  Glyma.08G078700.1       149.0     156.0   

      site_Strand  site_Score          site_Sequence  
0               +       37.74  CTCTCTCTCTCTCTCTCTCTC  
1            

In [40]:
def find_overlapping_intervals(start_pos, end_pos):
    if len(start_pos) == 0:
        return []
    
    # Create DataFrame from the input lists
    intervals = pd.DataFrame({
        'Start': start_pos,
        'End': end_pos
    })
    
    # Sort intervals by start position
    intervals = intervals.sort_values(by='Start').reset_index(drop=True)
    
    results = []
    current_start = intervals.loc[0, 'Start']
    current_end = intervals.loc[0, 'End']
    
    for i in range(1, len(intervals)):
        next_start = intervals.loc[i, 'Start']
        next_end = intervals.loc[i, 'End']
        
        if next_start <= current_end:
            # Extend the current interval
            current_end = max(current_end, next_end)
        else:
            # Record the non-overlapping interval
            results.append((current_start, current_end))
            current_start = next_start
            current_end = next_end
    
    # Append the last interval
    results.append((current_start, current_end))
    
    return results


In [42]:
# Initialize output list
output = []

# Loop through each unique gene-TF pair
for (gene, tf), subset_df in df_excel.groupby(['sequence_name', 'motif_alt_id']):
    start_positions = subset_df['start'].tolist()
    end_positions = subset_df['stop'].tolist()
    
    # Find overlapping intervals
    overlapping_intervals = find_overlapping_intervals(start_positions, end_positions)
    
    # Add results to output
    for start, end in overlapping_intervals:
        output.append([gene, tf, start, end])

# Convert to DataFrame
output_df = pd.DataFrame(output, columns=['Gene', 'TF', 'Start', 'End'])

# Output to .tsv and .xlsx files
output_df.to_csv('output_data_fimo.tsv', sep='\t', index=False)
output_df.to_excel('output_data_fimo.xlsx', index=False)

# Display the refined dataset
print("Refined Data:\n", output_df)

Refined Data:
                     Gene              TF   Start     End
0      Glyma.01G043300.1           AHL12   601.0   608.0
1      Glyma.01G043300.1           AHL12   910.0   917.0
2      Glyma.01G043300.1           AHL25   601.0   608.0
3      Glyma.01G043300.1           AHL25   910.0   917.0
4      Glyma.01G043300.1           ARF14    17.0    34.0
...                  ...             ...     ...     ...
48985  Glyma.20G163200.1  Zm00001d049364   292.0   307.0
48986  Glyma.20G163200.1  Zm00001d049364  1878.0  1899.0
48987  Glyma.20G163200.1         bHLH130  1164.0  1174.0
48988  Glyma.20G163200.1         bHLH145  1874.0  1893.0
48989  Glyma.20G163200.1          bHLH77  1867.0  1891.0

[48990 rows x 4 columns]


In [43]:
#contingency table
df_play = pd.read_csv('output_data_fimo.tsv',delimiter='\t')
df_play.fillna('NA',inplace=True)
cont_table_fimo = pd.crosstab(df_play['Gene'],df_play['TF'])#, margins=True)
print(cont_table_fimo)
print(df_play)
cont_table_fimo.to_csv('cont_table_fimo.tsv',sep='\t')
cont_table_fimo.to_excel('cont_table_fimo.xlsx',engine='openpyxl')

TF                 ABF2  ABI3  ABI5  ABR1  AHL12  AHL25  ANL2  AP3  ARF14  \
Gene                                                                        
Glyma.01G043300.1     0     0     0     0      2      2     0    0      1   
Glyma.01G053800.1     1     1     1     0      0      0     1    0      2   
Glyma.01G056800.1     1     2     1     0      1      1     0    0      0   
Glyma.01G128100.1     0     0     0     1      0      0     0    0      0   
Glyma.01G189100.1     0     1     0     1      1      1     1    0      2   
...                 ...   ...   ...   ...    ...    ...   ...  ...    ...   
Glyma.19G221700.1     1     1     1     3      0      0     1    0      2   
Glyma.19G254800.1     0     0     0     0      0      0     0    0      3   
Glyma.20G028000.1     0     1     0     0      4      4     0    0      1   
Glyma.20G030500.1     0     2     0     1      1      1     0    0      1   
Glyma.20G163200.1     1     1     1     0      0      0     1    0      2   

In [44]:
import numpy as np
from scipy.stats import spearmanr

# Load your contingency table, skipping the first column (gene names)
# Replace 'contingency_table.csv' with your actual file
contingency_table = pd.read_excel('cont_table_fimo.xlsx', index_col=0)

# Drop the first column if it's the gene names column
tf_table = contingency_table.iloc[:, 1:]  # Exclude the first column

# Define the columns of interest
columns_of_interest = ['CAI', 'rENC']

# Prepare to store results
significant_correlations = []

# Iterate through each column of interest
for col in columns_of_interest:
    for tf in tf_table.columns:
        if tf != col:
            # Calculate Spearman's rank correlation and p-value
            corr, p_value = spearmanr(tf_table[col], tf_table[tf])
            
            # Filter based on statistical significance (e.g., p < 0.05)
            if p_value < 0.05:
                significant_correlations.append((col, tf, corr, p_value))

# Convert the results to a DataFrame for better readability
significant_df = pd.DataFrame(significant_correlations, columns=['Column_of_Interest', 'TF', 'Correlation', 'P-Value'])

# Sort by the strength of the correlation
significant_df = significant_df.sort_values(by='Correlation', ascending=False)

# Save the significant correlations to a file
significant_df.to_csv('significant_cai_nc_correlations_spearman_fimo.csv', index=False)

# Display the top few results
print(significant_df.head())


   Column_of_Interest     TF  Correlation       P-Value
21                CAI   rENC     0.513522  1.685391e-13
37               rENC    CAI     0.513522  1.685391e-13
5                 CAI   ERF6     0.287729  8.982949e-05
33               rENC  TCP22     0.229307  1.958763e-03
32               rENC  TCP21     0.216852  3.455789e-03


In [45]:
import pandas as pd
from scipy.stats import spearmanr

# Load your contingency table, skipping the first column (gene names)
# Replace 'contingency_table.xlsx' with your actual file
contingency_table = pd.read_excel('cont_table_fimo.xlsx', index_col=0)

# Drop the first column if it's the gene names column
tf_table = contingency_table.iloc[:, 1:]  # Exclude the first column

# Define the columns of interest
columns_of_interest = ['rENC', 'CAI']

# Prepare dictionaries to store significant correlations
significant_enc = {}
significant_cai = {}

# Function to calculate significant correlations
def calculate_significant_correlations(column):
    significant = {}
    for tf in tf_table.columns:
        if tf != column:
            corr, p_value = spearmanr(tf_table[column], tf_table[tf])
            if p_value < 0.05:
                significant[tf] = (corr, p_value)
    return significant

# Calculate significant correlations with ENC
significant_enc = calculate_significant_correlations('rENC')

# Calculate significant correlations with CAI
significant_cai = calculate_significant_correlations('CAI')

# Find common TFs that are significant with both ENC and CAI
common_tfs = set(significant_enc.keys()).intersection(set(significant_cai.keys()))

# Prepare the results
common_tfs_results = []

for tf in common_tfs:
    enc_corr, enc_p_value = significant_enc[tf]
    cai_corr, cai_p_value = significant_cai[tf]
    common_tfs_results.append((tf, enc_corr, enc_p_value, cai_corr, cai_p_value))

# Convert the results to a DataFrame for better readability
common_tfs_df = pd.DataFrame(common_tfs_results, columns=['TF', 'ENC_Correlation', 'ENC_P-Value', 'CAI_Correlation', 'CAI_P-Value'])

# Sort by the strength of the correlation with ENC
common_tfs_df = common_tfs_df.sort_values(by='ENC_Correlation', ascending=False)

# Save the results to a file
common_tfs_df.to_csv('common_tfs_with_significant_correlations_fimo.csv', index=False)

# Display the results
print(common_tfs_df)


       TF  ENC_Correlation  ENC_P-Value  CAI_Correlation   CAI_P-Value
0  NAC031          0.16194     0.029865         0.157119  3.517100e-02
1     ENC         -1.00000     0.000000        -0.513522  1.685391e-13
